# Week 3: Transfer Learning
TL allows re-using existing model and learnt features just by downloading an open source model and use those params as a starting point, and train your model a bit more on a smaller dataset.

## Understanding TL concepts
Problem of previous week DS is that training data was very small, and there is only so many common features that can be extracted (even with image augmentation).
Instead of building a model from scratch, we can use an existing model that is trained on far more data and use the features that that model learned. You can use the features that were learned from large datasets that you may not have access to.
You can choose to re-train some of the lower layers, because they might be too specialized for the images at hand.

In the next few videos you'll be using this [notebook](https://colab.research.google.com/github/lmoroney/dlaicourse/blob/master/Course%202%20-%20Part%206%20-%20Lesson%203%20-%20Notebook.ipynb) to explore transfer learning.
For more on how to freeze/lock layers (`layer.trainable = false`), explore the documentation, which includes an example using MobileNet architecture [Transfer learning and fine-tuning](https://www.tensorflow.org/tutorials/images/transfer_learning)

In [ ]:
# Coding transfer learning from the inception mode
import os

# We will use Keras Layer API to peek at the layers and understand which ones we want to use and which ones to retrain
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5' # pretrained weights of inception Neural Network
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                include_top=False, # do not use built-in weights, ignore fully connected layer at the top and get straight to convolutions
                                weights=None)
pre_trained_model.load_weights(local_weights_file)

# Iterate through layers and lock them, saying that they are not gonna be trainable
for layer in pre_trained_model.layers:
    layer.trainable = False

pre_trained_model.summary() # lost of data!

# Coding your own model with transferred features
last_layer = pre_trained_model.get_layer('mixed7') # all of the layers have name, so we can get any layer by name
last_output = last_layer.output # take the output of the layer

from tensorflow.keras.optimizers import RMSprop
# Define our new model, taking `last_output` layer from inception model
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(1, activation='sigmoid')(x)
x = layers.Dropout(0.2)(x) # drop out 20% of the neurons

model = Model(pre_trained_model.input, x)
model.compile(optimizer=RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

# augment images with image generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# get training data from generator:
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join( base_dir, 'train')
train_generator = train_datagen.flow_from_directory(
    train_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150,150)
)

# train the network:
validation_generator = None #TODO: set it up
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    steps_per_epoch=100,
    epochs=100,
    validation_steps=50,
    verbose=2
)

## Using dropouts
Dropout is a regularization technique that prevents over fitting
The idea behind Dropouts is that they remove a random number of neurons in your neural network.
This works very well for two reasons: The first is that neighboring neurons often end up with similar weights, which can lead to overfitting, so dropping some out at random can remove this.
The second is that often a neuron can over-weigh the input from a neuron in the previous layer, and can over specialize as a result.
Thus, dropping out can break the neural network out of this potential bad habit!

Check out Andrew's terrific video explaining dropouts [here](https://www.youtube.com/watch?v=ARq74QuavAo)